In [3]:
# from datetime import date, timedelta

# # Remove this before running Data Factory Pipeline
# start_date = date.today() - timedelta(1)

# silver_adls = "abfss://silver@synonlypunya.dfs.core.windows.net/"
# gold_adls = "abfss://gold@synonlypunya.dfs.core.windows.net/"

# silver_data = f"{silver_adls}earthquake_events_silver/"

StatementMeta(synapseonly2, 2, 2, Finished, Available, Finished)

In [ ]:
bronze_output=""
silver_output=""

In [ ]:
import json
bronze_data=json.loads(bronze_output)
start_date=bronze_data.get("start_date")
silver_adls=bronze_data.get("silver_adls")
gold_adls=bronze_data.get("gold_adls")
silver_data=f"{silver_adls}earthquake_events_silver/"

In [7]:
from pyspark.sql.functions import col, when, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
from datetime import date, timedelta

StatementMeta(synapseonly2, 2, 6, Finished, Available, Finished)

In [9]:
df=spark.read.parquet(silver_data).filter(col('time')>start_date)

StatementMeta(synapseonly2, 2, 8, Finished, Available, Finished)

In [10]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

StatementMeta(synapseonly2, 2, 9, Finished, Available, Finished)

In [11]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(synapseonly2, 2, 10, Finished, Available, Finished)

In [12]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(synapseonly2, 2, 11, Finished, Available, Finished)

In [13]:
# adding significance classification
df_with_location_sig_class = \
                            df.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(synapseonly2, 2, 12, Finished, Available, Finished)

In [14]:
# Save the transformed DataFrame to the Silver container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

StatementMeta(synapseonly2, 2, 13, Finished, Available, Finished)

In [15]:
# Append DataFrame to Silver container in Parquet format
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)

StatementMeta(synapseonly2, 2, 14, Finished, Available, Finished)